In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

# Import and Build Model

In [ ]:
from deeprte.experiment import Trainer
import jax
import haiku as hk

In [ ]:
source_dir = "/workspaces/deeprte/rte_data/rte_data/matlab/train-delta"
data_name_list = ["train_delta_1.mat","train_delta_2.mat"]

In [ ]:
from deeprte.config import get_config
config = get_config()
config = config.experiment_kwargs.config
config.dataset.source_dir = source_dir
config.dataset.data_name_list = data_name_list
rng = hk.PRNGSequence(jax.random.PRNGKey(42))
e = Trainer(mode = "train", init_rng=0, config = config)

# Data Shape Visualization

In [ ]:
import tensorflow as tf
tf.nest.map_structure(lambda x: x.shape, (e.tf_data))

## Train Input

In [ ]:
train_input = e._build_train_input()
tf.nest.map_structure(lambda x: x.shape, next(train_input))

## Val Input

In [ ]:
val_input = e._build_eval_input()
tf.nest.map_structure(lambda x: x.shape, next(val_input))

## Dummy Input for Model Init

In [ ]:
ds = e._build_dummy_input()

In [ ]:
tf.nest.map_structure(lambda x: x.shape, ds)

# Test Train Step

In [ ]:
from jaxline import utils as jl_utils
init_rng = jl_utils.bcast_local_devices(next(rng))
e._initialize_training()
iterator=tf.data.Dataset.from_tensors(ds).as_numpy_iterator()
e._train_input = iterator

In [ ]:
scalars = e.step(jl_utils.bcast_local_devices(0), jl_utils.bcast_local_devices(next(rng)))

In [ ]:
scalars

# Test Evaluation

In [ ]:
e._eval_input = iterator

In [ ]:
metrics = e.evaluate(jl_utils.bcast_local_devices(0), jl_utils.bcast_local_devices(next(rng)))

In [ ]:
metrics